In [1]:
import pandas as pd
import pymongo as pm

db = pm.MongoClient().scraper_meta

In [54]:
# res_tissues.set_index('series').to_pickle('../data/geo-annotation/series.tissue.all.res1.pickle')

In [2]:
res_tissues = pd.read_pickle('../data/geo-annotation/series.tissue.all.res1.pickle').reset_index()
res_tissues.head()

,series,collapsed
0,GSE10,BTO:0001177
1,GSE10003,BTO:0000762
2,GSE10007,BTO:0001573
3,GSE10040,BTO:0001025
4,GSE10041,BTO:0000089


In [3]:
series_disease = pd.read_pickle('../data/geo-annotation/series_disease.pickle')
series_disease.head()

,series,doid,disease,samples_count,platforms
0,GSE3,DOID:4467,renal clear cell carcinoma,342,"[GPL9, GPL10]"
2,GSE16,DOID:1612,breast cancer,25,[GPL28]
3,GSE17,DOID:440,neuromuscular disease,2,[GPL4]
5,GSE31,DOID:161,keratosis,5,[GPL4]
9,GSE49,DOID:1612,breast cancer,10,[GPL179]


In [4]:
_t = db.series.find({
        'accession': {'$in': res_tissues.series.unique().tolist()}
    },{
        '_id': 0,
        'accession': 1,
        'samples': 1,
        'platforms': 1
    })

series_tissue = (
    pd.DataFrame.from_records([(c['accession'], c['samples'], c['platforms']) for c in _t],
                             columns=['series', 'samples', 'platforms'])
    .set_index('series')
    .join(res_tissues.set_index('series'))
    .reset_index()
    .rename(columns=dict(collapsed='tissue_id'))
#     .assign(samples=lambda d: d['samples'].map(len))
)a
series_tissue[:5]

,series,samples,platforms,tissue_id
0,GSE10,"[GSM571, GSM572, GSM573, GSM574]",[GPL4],BTO:0001177
1,GSE10003,"[GSM265549, GSM265550, GSM265551, GSM265552]",[GPL6104],BTO:0000762
2,GSE10007,"[GSM252886, GSM252887, GSM252888, GSM252889]",[GPL2005],BTO:0001573
3,GSE10040,"[GSM259205, GSM259206, GSM259207, GSM259208, G...",[GPL571],BTO:0001025
4,GSE10041,"[GSM253663, GSM253664, GSM253665, GSM253666, G...",[GPL570],BTO:0000089


## Статистика по платформам

In [5]:
(
    series_tissue
    .assign(platforms=lambda d: d['platforms'].map(','.join))
    .assign(samples=lambda d: d['samples'].map(len))
    .groupby('platforms')
    .samples
    .sum()
    .to_frame()
    .sort('samples', ascending=False)
    [:15]
)

,samples
platforms,
GPL570,42367
GPL10558,19603
GPL96,13735
GPL13534,12182
GPL6244,10499
GPL6947,9911
GPL8490,8846
GPL6480,7696
GPL6801,6574


## Статистика по тканям

In [19]:
import sys
sys.path.append('..')

In [139]:
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad

reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph, search_item, search_term
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)
pd.set_option('display.max_rows', 200)

In [21]:
tissue_ontology = read_ontology('../data/geo-annotation/brenda-tissue-ontology.obo',
                         exclude_duplicates=True,
                         subgraph='BTO:0001489')

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 5659
Number of edges: 6430
Average in degree:   1.1362
Average out degree:   1.1362


In [94]:
disease_ontology = read_ontology('../data/geo-annotation/doid-patched.obo',
                         exclude_duplicates=True)

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 6595
Number of edges: 6594
Average in degree:   0.9998
Average out degree:   0.9998


In [10]:
@pymonad.curry
def name(ontology, node):
    return '{} ({})'.format(ontology.meta[node].id, ontology.meta[node].name)

In [153]:
def cell():
    level = 2
    def descendants_level(graph, node, level):
        if level == 0:
            return [node]
        else:
            successors = graph.successors(node)
            res = []
            for succ in successors:
                res += descendants_level(graph, succ, level - 1)
            return res    
#     higer_nodes = [n for n in descendants_level(disease_ontology.graph, root_node, level)] + [root_node]
    len(higer_nodes)
    
    return len(descendants_level(disease_ontology.graph, 'DOID:4', 3))
cell()

263

In [13]:
def level_counts_tissue(ontology, root_node, level):
    def descendants_level(graph, node, level):
        if level == 0:
            return [node]
        else:
            successors = graph.successors(node)
            res = []
            for succ in successors:
                res += descendants_level(graph, succ, level - 1)
            return res    
    higer_nodes = [n for n in descendants_level(ontology.graph, root_node, level)] + [root_node]
    len(higer_nodes)
    
    @pymonad.curry
    def get_top_disease(higher_nodes, doid):
        res =[node for node in higer_nodes if ontology.has_path(node, doid)]
        if res:
            return res[0]
        else:
            return np.nan
    
    return (
        series_tissue
        .assign(top_doid=series_tissue.tissue_id.map(get_top_disease(higer_nodes)))
        .assign(samples=lambda d: d['samples'].map(len))
        .groupby('top_doid')
        .samples
        .sum()
        .to_frame()
        .sort('samples', ascending=False)
        .assign(name=lambda d: d.index.map(name(ontology)))
    )

### Верхний уровень

In [14]:
level_counts_tissue(tissue_ontology, 'BTO:0001489', 1)

,samples,name
top_doid,,
BTO:0000570,150542,BTO:0000570 (hematopoietic system)
BTO:0000522,83248,BTO:0000522 (gland)
BTO:0000634,43865,BTO:0000634 (integument)
BTO:0001491,25316,BTO:0001491 (viscus)
BTO:0000203,22749,BTO:0000203 (respiratory system)
BTO:0000282,19692,BTO:0000282 (head)
BTO:0000421,8825,BTO:0000421 (connective tissue)
BTO:0003091,6764,BTO:0003091 (urogenital system)
BTO:0001486,5017,BTO:0001486 (skeletal system)


### hematopoietic system

In [15]:
level_counts_tissue(tissue_ontology, 'BTO:0000570', 1)

,samples,name
top_doid,,
BTO:0000089,145485,BTO:0000089 (blood)
BTO:0001239,5717,BTO:0001239 (serum)
BTO:0000855,781,BTO:0000855 (lymph)
BTO:0000753,531,BTO:0000753 (lymphoid tissue)
BTO:0000752,78,BTO:0000752 (lymph vessel)


## Ткань - заболевание

In [16]:
series_tissue.head()

,series,samples,platforms,tissue_id
0,GSE10,"[GSM571, GSM572, GSM573, GSM574]",[GPL4],BTO:0001177
1,GSE10003,"[GSM265549, GSM265550, GSM265551, GSM265552]",[GPL6104],BTO:0000762
2,GSE10007,"[GSM252886, GSM252887, GSM252888, GSM252889]",[GPL2005],BTO:0001573
3,GSE10040,"[GSM259205, GSM259206, GSM259207, GSM259208, G...",[GPL571],BTO:0001025
4,GSE10041,"[GSM253663, GSM253664, GSM253665, GSM253666, G...",[GPL570],BTO:0000089


In [17]:
series_disease.head()

,series,doid,disease,samples_count,platforms
0,GSE3,DOID:4467,renal clear cell carcinoma,342,"[GPL9, GPL10]"
2,GSE16,DOID:1612,breast cancer,25,[GPL28]
3,GSE17,DOID:440,neuromuscular disease,2,[GPL4]
5,GSE31,DOID:161,keratosis,5,[GPL4]
9,GSE49,DOID:1612,breast cancer,10,[GPL179]


In [20]:
series_tissue_disease = (
    pd.merge((
                series_tissue
                .assign(samples_count=lambda d: d['samples'].map(len))
             )
             , series_disease, 
            left_on='series', right_on='series',
            how='outer')
    .assign(samples_count=lambda d: d['samples_count_x'].combine(d['samples_count_y'], func=lambda x, y: y if not pd.isnull(y) else x))
    [['series', 'samples_count', 'doid', 'tissue_id']]
)

In [98]:
def map_higer_level(ontology, root_node):
    level = 1
    def descendants_level(graph, node, level):
        if level == 0:
            return [node]
        else:
            successors = graph.successors(node)
            res = []
            for succ in successors:
                res += descendants_level(graph, succ, level - 1)
            return res    
    higer_nodes = [n for n in descendants_level(ontology.graph, root_node, level)] + [root_node]
    len(higer_nodes)
    
    @pymonad.curry
    def get_top_disease(higher_nodes, doid):
        res =[node for node in higer_nodes if ontology.has_path(node, doid)]
        if res:
            return res[0]
        else:
#             raise Exception('No higer node for {}'.format(doid))
            return np.nan
    
    return get_top_disease(higer_nodes)

In [96]:
@pymonad.curry
def name(ontology, node):
    if pd.isnull(node):
        return np.nan
    item = ontology.meta[node]
    return "{} ({})".format(
        item.name, item.id
    )

In [23]:
(name(disease_ontology) * map_higer_level(disease_ontology, 'DOID:4'))('DOID:117')

'disease of cellular proliferation (DOID:14566)'

### Верхние уровни онтоголии болезней

In [92]:
def create_mapping(ontology, ids, root_node):
    mapf = name(ontology) * map_higer_level(ontology, root_node)
    map_dict = dict((id_, mapf(id_)) for id_ in ids)
    return lambda node: map_dict.get(node, np.nan)

In [99]:
mapping_top_disease = create_mapping(disease_ontology, series_disease.doid.unique(), 'DOID:4')
mapping_cancer_disease = create_mapping(disease_ontology, series_disease.doid.unique(), 'DOID:0050686')
mapping_top_tissue = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001489')

In [100]:
mapping_hematopoietic_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000570')
mapping_gland = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000522')
mapping_integument = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000634')
mapping_viscus = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001491')
mapping_respiratory_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000203')
mapping_head = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000282')
mapping_connective_tissue = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000421')
mapping_urogenital_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0003091')
mapping_skeletal_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001486')
mapping_cardiovascular_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000088')
mapping_nervous_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001484')
mapping_adult_stem_cell = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0002666')
mapping_sense_organ = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000202')
mapping_body_wall = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0000139')
mapping_trunk = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001493')
mapping_limb = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001492')
mapping_primary_cell = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001413')
mapping_muscular_system = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001485')

In [101]:
# endocrine gland (BTO:0001488)
mapping_endocrine_gland = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001488')

In [113]:
mapping_skin = create_mapping(tissue_ontology, series_tissue.tissue_id.unique(), 'BTO:0001253')

In [26]:
( 
    series_disease
    .assign(doid=series_disease.doid.map(mapping_top_disease))
    .groupby('doid')
    .samples_count
    .sum()
    .to_frame()
)

,samples_count
doid,
disease by infectious agent (DOID:0050117),7642
disease of anatomical entity (DOID:7),58615
disease of cellular proliferation (DOID:14566),308095
disease of mental health (DOID:150),13001
disease of metabolism (DOID:0014667),8676
genetic disease (DOID:630),2381


### Раки

In [28]:
( 
    series_disease
    .assign(doid=series_disease.doid.map(mapping_cancer_disease))
    .dropna()
    .groupby('doid')
    .samples_count
    .sum()
    .to_frame()
)

,samples_count
doid,
endocrine gland cancer (DOID:170),4984
gastrointestinal system cancer (DOID:3119),47896
head and neck cancer (DOID:11934),3145
immune system cancer (DOID:0060083),65420
integumentary system cancer (DOID:0060122),1247
musculoskeletal system cancer (DOID:0060100),7447
nervous system cancer (DOID:3093),12362
peritoneum cancer (DOID:1725),41
reproductive organ cancer (DOID:193),34357


### Верхний уровень тканей

In [108]:
series_tissue.samples.map(len).sum()

376794

In [31]:
#'BTO:0001489
( 
    series_tissue
    .assign(tissue_id=series_tissue.tissue_id.map(mapping_top_tissue))
    .assign(samples=lambda d: d.samples.map(len))
    .dropna()
    .groupby('tissue_id')
    .samples
    .sum()
    .to_frame()
    .sort('samples', ascending=False)
)

,samples
tissue_id,
hematopoietic system (BTO:0000570),150542
gland (BTO:0000522),83248
integument (BTO:0000634),43865
viscus (BTO:0001491),25316
respiratory system (BTO:0000203),22749
head (BTO:0000282),19692
connective tissue (BTO:0000421),8825
urogenital system (BTO:0003091),6764
skeletal system (BTO:0001486),5017


### Верхние ткани, раки

In [50]:
_t = (
    series_tissue_disease
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_top_tissue))
    .assign(doid=lambda d: d.doid.map(mapping_cancer_disease))
    .dropna()
    .groupby(['tissue_id', 'doid'])
    .samples_count
    .sum()
    .to_frame()
)
_d = (
    _t
    .reset_index(level=1)
    .drop('doid', axis=1)
    .groupby(level=0)
    .samples_count
    .sum()
    .to_frame()
    .rename(columns={'samples_count': 'samples_total'})

)
( 
    pd.merge(_d, _t, left_index=True, right_index=True)
    .sort(['samples_total', 'samples_count'], ascending=False)
    .drop('samples_total', axis=1)
)

samples_count
tissue_id                           doid                                                       
gland (BTO:0000522)                 thoracic cancer (DOID:5093)                           30429
                                    reproductive organ cancer (DOID:193)                  11865
                                    gastrointestinal system cancer (DOID:3119)             8817
                                    endocrine gland cancer (DOID:170)                       604
                                    head and neck cancer (DOID:11934)                        76
                                    immune system cancer (DOID:0060083)                      18
                                    nervous system cancer (DOID:3093)                        11
hematopoietic system (BTO:0000570)  immune system cancer (DOID:0060083)                   12643
                                    gastrointestinal system cancer (DOID:3119)             1547
                                    reproductive organ cancer (DOID:193)                   1499
                                    thoracic cancer (DOID:5093)                             892
                                    nervous system cancer (DOID:3093)                       582
                                    respiratory system cancer (DOID:0050615)                483
                                    urinary system cancer (DOID:3996)                       344
                                    head and neck cancer (DOID:11934)                       241
                                    musculoskeletal system cancer (DOID:0060100)             89
                                    endocrine gland cancer (DOID:170)                        40
                                    integumentary system cancer (DOID:0060122)               16
integument (BTO:0000634)            thoracic cancer (DOID:5093)                            6165
                                    gastrointestinal system cancer (DOID:3119)             4363
                                    respiratory system cancer (DOID:0050615)               1667
                                    reproductive organ cancer (DOID:193)                    463
                                    integumentary system cancer (DOID:0060122)              460
                                    immune system cancer (DOID:0060083)                     318
                                    musculoskeletal system cancer (DOID:0060100)            196
                                    urinary system cancer (DOID:3996)                       111
                                    head and neck cancer (DOID:11934)                        76
                                    endocrine gland cancer (DOID:170)                        41
                                    nervous system cancer (DOID:3093)                        30
respiratory system (BTO:0000203)    respiratory system cancer (DOID:0050615)              12284
                                    musculoskeletal system cancer (DOID:0060100)             69
                                    gastrointestinal system cancer (DOID:3119)               48
                                    nervous system cancer (DOID:3093)                        16
                                    urinary system cancer (DOID:3996)                        15
                                    thoracic cancer (DOID:5093)                               2
viscus (BTO:0001491)                gastrointestinal system cancer (DOID:3119)             5373
                                    urinary system cancer (DOID:3996)                      3603
                                    immune system cancer (DOID:0060083)                      77
                                    thoracic cancer (DOID:5093)                              46
                                    reproductive organ cancer (DOID:193)                     26
                                    endocrine glan

In [61]:
(
    series_tissue_disease
    .assign(tissue_id_h=lambda d: d.tissue_id.map(mapping_top_tissue))
    .assign(doid_h=lambda d: d.doid.map(mapping_cancer_disease))
    .set_index(['tissue_id_h', 'doid_h'])
    .assign(tissue_id=lambda d: d.tissue_id.map(name(tissue_ontology)))
    .assign(doid=lambda d: d.doid.map(name(disease_ontology)))    
    .dropna()
).loc['viscus (BTO:0001491)', 'urinary system cancer (DOID:3996)']
# _d.join(_t)

/usr/local/lib/python3.4/site-packages/pandas/core/index.py:5091: PerformanceWarning: indexing past lexsort depth may impact performance.
  PerformanceWarning)


series  samples_count                                    doid                                     tissue_id
tissue_id_h          doid_h                                                                                                                                          
viscus (BTO:0001491) urinary system cancer (DOID:3996)  GSE11904             21        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE11985             12        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE12105             24        renal cell carcinoma (DOID:4450)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE12114             27        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE12606             10        renal cell carcinoma (DOID:4450)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE13282             21        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE14762             22        renal cell carcinoma (DOID:4450)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE15193             12        renal cell carcinoma (DOID:4450)                       achn cell (BTO:0003846)
                     urinary system cancer (DOID:3996)  GSE16441             68        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE16449             70        renal cell carcinoma (DOID:4450)                renal clear cell (BTO:0004098)
                     urinary system cancer (DOID:3996)  GSE17746             31  chromophobe adenocarcinoma (DOID:4471)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE17816             45        renal cell carcinoma (DOID:4450)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE19568              4     urinary bladder cancer (DOID:11054)                       t-24 cell (BTO:0001345)
                     urinary system cancer (DOID:3996)  GSE19712              1     urinary bladder cancer (DOID:11054)                         bladder (BTO:0000123)
                     urinary system cancer (DOID:3996)  GSE19716              5     urinary bladder cancer (DOID:11054)     urinary bladder cancer cell (BTO:0000362)
                     urinary system cancer (DOID:3996)  GSE19717              4     urinary bladder cancer (DOID:11054)     urinary bladder cancer cell (BTO:0000362)
                     urinary system cancer (DOID:3996)  GSE19915            285     urinary bladder cancer (DOID:11054)                 urinary bladder (BTO:0001418)
                     urinary system cancer (DOID:3996)  GSE21816             18                kidney cancer (DOID:263)                          kidney (BTO:0000671)
                     urinary system cancer (DOID:3996)  GSE22256              2        renal cell carcinoma (DOID:4450)              kidney cancer cell (BTO:0000680)
                     urinary system cancer (DOID:3996)  GSE22563             22        renal cell carcinoma (DOID:4450)  renal cell carcinoma cell line (BTO:0000383)
                     urinary system cancer (DOID:3996)  GSE23085             20        renal cell carcinoma (DOID:4450)                    renal cortex (BTO:0001166)
                     urinary system cancer (DOID:3996)  GSE23614              4                kidney cancer (DOID:263)          renal cancer cell line (BTO:0003192)
                     urinary system cancer (DOID:3996)  GSE23690          

In [5]:
res_series = pd.read_pickle('../data/geo-annotation/series.age.res.1.py3.pickle')

In [9]:
series_tissue[series_tissue.series.isin(series_age_nottissue)]

,series,samples,platforms,tissue_id


In [83]:
_t = pd.merge(series_tissue, res_series, how='inner')
_t[_t.tissue_id.isin(higer_nodes)].series.unique()

array(['GSE11622', 'GSE18098', 'GSE18876', 'GSE22396', 'GSE27949',
       'GSE27951', 'GSE34460', 'GSE34535', 'GSE34536', 'GSE35411',
       'GSE37223', 'GSE37689', 'GSE42432', 'GSE42902', 'GSE45516',
       'GSE47162', 'GSE47511', 'GSE47512', 'GSE47513', 'GSE51518',
       'GSE51519', 'GSE53424', 'GSE53425', 'GSE53426', 'GSE53431',
       'GSE54638', 'GSE55664', 'GSE57225', 'GSE58121', 'GSE62354',
       'GSE66511', 'GSE67098', 'GSE6710', 'GSE67762'], dtype=object)

In [87]:
skin_age_series = [
    'GSE11622', 'GSE18098', 'GSE18876', 'GSE22396', 'GSE27949',
       'GSE27951', 'GSE34460', 'GSE34535', 'GSE34536', 'GSE35411',
       'GSE37223', 'GSE37689', 'GSE42432', 'GSE42902', 'GSE45516',
       'GSE47162', 'GSE47511', 'GSE47512', 'GSE47513', 'GSE51518',
       'GSE51519', 'GSE53424', 'GSE53425', 'GSE53426', 'GSE53431',
       'GSE54638', 'GSE55664', 'GSE57225', 'GSE58121', 'GSE62354',
       'GSE66511', 'GSE67098', 'GSE6710', 'GSE67762'
]

_q = [(c['accession'], p) for c in db.series.find({'accession': {'$in': skin_age_series}},
               {'_id': 0, 'accession' :1, 'platforms': 1})
 for p in c['platforms']]

_e = pd.DataFrame.from_records(_q, columns=['series', 'platform'])
_e.platform.value_counts()

GPL10558    5
GPL570      5
GPL14550    3
GPL15019    3
GPL13534    2
GPL4133     2
GPL5175     2
GPL13829    2
GPL11154    2
GPL11532    1
GPL342      1
GPL96       1
GPL10379    1
GPL5477     1
GPL2879     1
GPL19128    1
GPL6480     1
GPL17148    1
GPL341      1
GPL13273    1
GPL16288    1
GPL10335    1
GPL6883     1
GPL13607    1
dtype: int64

In [6]:
_t = pd.merge(series_tissue, res_series, how='right')
series_age_nottissue = _t[_t.tissue_id.isnull()].series.to_frame().drop_duplicates()
series_age_nottissue[:10]

,series
51661,GSE36942
51703,GSE40738
51880,GSE40801
51882,GSE40870
51930,GSE40871
51978,GSE40911
52022,GSE40914
52098,GSE40912
52130,GSE41223
52174,GSE41233


In [54]:
_c = db.samples.find({'series': {'$in': series_age_nottissue.series.tolist()},
                      'organism': 'Homo sapiens'},
               {'_id': 0, 'accession': 1, 'series': 1})

sample_series_age_no_tissue = pd.DataFrame.from_records([(x['accession'], series) 
                                                         for x in _c 
                                                         for series in x['series']], 
                                                        columns=['sample', 'series'])
samples_to_annotate = pd.merge(series_age_nottissue, sample_series_age_no_tissue)['sample'].unique().tolist()
samples_to_annotate[:10]

['GSM906744',
 'GSM906745',
 'GSM906746',
 'GSM906748',
 'GSM906749',
 'GSM906747',
 'GSM906750',
 'GSM906751',
 'GSM906752',
 'GSM906753']

In [55]:
res_series[res_series.accession.isin(samples_to_annotate)].shape

(56924, 13)

In [56]:
len(samples_to_annotate), len(sample_series_age_no_tissue['sample'].unique())

(60660, 60660)

In [79]:
def descendants_level(graph, node, level):
    if level == 0:
        return [node]
    else:
        successors = graph.successors(node)
        res = []
        res.append(node)
        for succ in successors:
            res += descendants_level(graph, succ, level - 1)
        return res    
root_node = 'BTO:0001253'
higer_nodes = [n for n in descendants_level(tissue_ontology.graph, root_node, 902)] + [root_node]
len(higer_nodes)

455

In [130]:
import elasticsearch
es = elasticsearch.Elasticsearch()
samples_res_1 = pd.DataFrame.from_records( [(id_, node) 
                                            for node in higer_nodes 
                                            for id_ in search_item(client=es, 
                                                                     item=tissue_ontology.meta[node], 
                                                                     index='samples', 
                                                                     ids=[int(s[3:]) for s in samples_to_annotate])],
                                                                     columns=['sample_id', 'tissue_id']
)



In [131]:
samples_res_1.sample_id.unique().shape

(1302,)

In [66]:
@pymonad.curry
def names(ontology, df):
    return df.applymap(lambda n: '{} ({})'.format(ontology.meta[n].name, ontology.meta[n].id))

In [59]:
names(tissue_ontology)(samples_res_1.set_index('sample_id')).tissue_id.value_counts().sum()

1934

In [133]:
_t = (
    samples_res_1
    .groupby('sample_id')
    .tissue_id
    .apply(set)
    .map(list)
    .to_frame()
)

names(tissue_ontology)(_t[_t.tissue_id.map(len) == 1].tissue_id.map(lambda x: x[0]).to_frame()).tissue_id.value_counts()
_f = (_t[_t.tissue_id.map(len) == 1].tissue_id.map(lambda x: x[0]).to_frame())
to_find_norm_samples1 = _f[_f.tissue_id.isin(['BTO:0001253', 'BTO:0000404', 'BTO:0000294'])].index.unique()
to_find_norm_samples1 = ['GSM{}'.format(x) for x in to_find_norm_samples1]
to_find_norm_samples1[:10]

['GSM72008',
 'GSM72009',
 'GSM72010',
 'GSM72011',
 'GSM72012',
 'GSM72013',
 'GSM72014',
 'GSM72015',
 'GSM134115',
 'GSM134116']

In [68]:
_t[_t.tissue_id.map(len) > 1].applymap(lambda d: [tissue_ontology.meta[n].name for n in d])

,tissue_id
sample_id,
172521,"[keratinocyte, keratinocyte cell line]"
344311,"[melanocyte, skin]"
384127,"[skin, basal cell]"
384128,"[skin, basal cell]"
590766,"[melanocyte, scalp]"
695345,"[melanocyte, melanocyte cell line]"
695346,"[melanocyte, melanocyte cell line]"
695351,"[melanocyte, melanocyte cell line]"
776959,"[dermal papilla, hair follicle, hair]"


In [102]:
series_age = pd.merge(series_tissue, res_series)
series_age.head()

,series,samples,platforms,tissue_id,rid,title,accession,description,age_str,age,detected,detect_crit,is_form_body,is_form_cell_line,datetime,source_name
0,GSE10072,"[GSM254625, GSM254626, GSM254627, GSM254628, G...",[GPL96],BTO:0000763,116807,Normal Lung_GT00006,GSM254626,Gene expression data,69,69,1,5,1,0,2015-10-01 00:20:34,Normal Lung Tissue
1,GSE10072,"[GSM254625, GSM254626, GSM254627, GSM254628, G...",[GPL96],BTO:0000763,116809,Normal Lung_GT00007,GSM254628,Gene expression data,68,68,1,5,1,0,2015-10-01 00:20:34,Normal Lung Tissue
2,GSE10072,"[GSM254625, GSM254626, GSM254627, GSM254628, G...",[GPL96],BTO:0000763,116813,Normal Lung_GT00059,GSM254632,Gene expression data,67,67,1,5,1,0,2015-10-01 00:20:34,Normal Lung Tissue
3,GSE10072,"[GSM254625, GSM254626, GSM254627, GSM254628, G...",[GPL96],BTO:0000763,116814,Normal Lung_GT00146,GSM254634,Gene expression data,71,71,1,5,1,0,2015-10-01 00:20:34,Normal Lung Tissue
4,GSE10072,"[GSM254625, GSM254626, GSM254627, GSM254628, G...",[GPL96],BTO:0000763,116816,Normal Lung_GT01001,GSM254635,Gene expression data,70,70,1,5,1,0,2015-10-01 00:20:34,Normal Lung Tissue


In [132]:
series_age.shape

(51661, 16)

In [125]:
(
    series_tissue
    .assign(samples=lambda d: d['samples'].map(len))
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_integument))
    .dropna()
    .groupby('tissue_id')
    .samples
    .sum()
    .to_frame().sort('samples', ascending=False)
)

,samples
tissue_id,
epithelium (BTO:0000416),23682
skin (BTO:0001253),13619
mucosa (BTO:0000886),6564


In [128]:
(
    series_tissue
    .assign(samples=lambda d: d['samples'].map(len))
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
    .dropna()
    .groupby('tissue_id')
    .samples
    .sum()
    .to_frame().sort('samples', ascending=False)
)

,samples
tissue_id,
skin (BTO:0001253),4729
skin fibroblast (BTO:0001255),2791
epidermis (BTO:0000404),2612
dermis (BTO:0000294),2059
skin cancer cell (BTO:0001286),1772
scalp (BTO:0001809),10


In [136]:
# to_find_norm_samples1
_g = (
    series_tissue
#     .assign(samples=lambda d: d['samples'].map(len))
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
    .dropna()
    .samples.tolist()
#     .groupby('tissue_id')
#     .samples
#     .sum()
#     .to_frame().sort('samples', ascending=False)
)

to_find_norm_samples2 = [sample for samples in _g for sample in samples]
to_find_norm_samples2[:10]

['GSM255774',
 'GSM255806',
 'GSM255807',
 'GSM255808',
 'GSM255809',
 'GSM255828',
 'GSM255829',
 'GSM255830',
 'GSM255831',
 'GSM255832']

In [137]:
len(to_find_norm_samples1), len(to_find_norm_samples2)

(507, 13973)

In [138]:
to_find_norm_samples =list(set(to_find_norm_samples1 + to_find_norm_samples2))
len(to_find_norm_samples)

13244

### Нормы для кож

In [144]:
norms = search_term(es=es,
            term=['healty', 'control', 'reference', 'normal'],
            index='samples',
            ids=[int(s[3:]) for s in to_find_norm_samples if not 'Accession list truncated' in s])

In [147]:
len(norms)/len(to_find_norm_samples)

0.3488372093023256

In [111]:
# cancer_top_ids = 
_t = (
    series_age
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_endocrine_gland))
    .assign(age_range=lambda d: pd.cut(d.age, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))
    .assign(_count=lambda d: 1)
    [['series', 'tissue_id', 'age', 'age_range', '_count']]
    .groupby(['tissue_id', 'age_range'])
    .sum()
    .dropna()
    ._count
    .to_frame()
#     .sum()
)

_d = (
    _t
    .reset_index('age_range')
    .drop('age_range', axis=1)
    .groupby(level=0)
    .sum()
    .rename(columns={'_count': '__count'})
)
(
    _t.join(_d)
    .sort(['__count', '_count'], ascending=False)
    .rename(columns={'_count': 'samples_count'})
    .drop('__count', axis=1)
)
# integument (BTO:0000634)


samples_count
tissue_id                               age_range               
female reproductive gland (BTO:0000254) (50, 60]            1686
                                        (40, 50]            1644
                                        (60, 70]            1236
                                        (30, 40]             859
                                        (70, 80]             561
                                        (20, 30]             171
                                        (80, 90]             142
                                        (0, 10]               77
                                        (10, 20]              22
                                        (90, 100]             19
digestive gland (BTO:0000345)           (60, 70]            1100
                                        (50, 60]            1045
                                        (40, 50]             750
                                        (70, 80]             452
                                        (30, 40]             442
                                        (0, 10]              328
                                        (20, 30]             221
                                        (10, 20]             160
                                        (80, 90]              32
male reproductive gland (BTO:0000080)   (60, 70]             380
                                        (50, 60]             276
                                        (70, 80]             148
                                        (10, 20]              69
                                        (40, 50]              57
                                        (30, 40]              33
                                        (20, 30]              27
                                        (80, 90]               6
thyroid gland (BTO:0001379)             (40, 50]             136
                                        (50, 60]             126
                                        (30, 40]             102
                                        (60, 70]              88
                                        (20, 30]              39
                                        (70, 80]              39
                                        (80, 90]               6
                                        (10, 20]               2
adrenal gland (BTO:0000047)             (20, 30]              17
                                        (40, 50]              15
                                        (30, 40]              14
                                        (10, 20]              12
                                        (50, 60]               9
                                        (60, 70]               5
                                        (0, 10]                1
                                        (70, 80]               1
hypophysis (BTO:0001073)                (50, 60]               8
                                        (40, 50]               7
                                        (30, 40]               6
                                        (60, 70]               6
                                        (70, 80]               2
                                        (10, 20]               1
                                        (80, 90]               1
                                        (90, 100]              1

In [114]:
# cancer_top_ids = 
_t = (
    series_age
#     .assign(tissue_id=lambda d: d.tissue_id.map(mapping_integument))
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
    .assign(age_range=lambda d: pd.cut(d.age, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))
    .assign(_count=lambda d: 1)
    [['series', 'tissue_id', 'age', 'age_range', '_count']]
    .groupby(['tissue_id', 'age_range'])
    .sum()
    .dropna()
    ._count
    .to_frame()
#     .sum()
)

_d = (
    _t
    .reset_index('age_range')
    .drop('age_range', axis=1)
    .groupby(level=0)
    .sum()
    .rename(columns={'_count': '__count'})
)
(
    _t.join(_d)
    .sort(['__count', '_count'], ascending=False)
    .rename(columns={'_count': 'samples_count'})
    .drop('__count', axis=1)
    .reset_index('age_range').drop('age_range', axis=1)
    .reset_index()
    .groupby('tissue_id')
    .samples_count
    .sum()
)
# integument (BTO:0000634)


tissue_id
dermis (BTO:0000294)             190
epidermis (BTO:0000404)          127
skin (BTO:0001253)               496
skin fibroblast (BTO:0001255)     83
Name: samples_count, dtype: float64

In [125]:
_t = (
    series_age
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
#     .assign(age_range=lambda d: pd.cut(d.age, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))
#     .assign(_count=lambda d: 1)
#     [['series', 'tissue_id', 'age', 'age_range', '_count']]
#     .groupby(['tissue_id', 'age_range'])
#     .sum()
    .dropna()
#     ._count
#     .to_frame()
#     .sum()
)
_t['accession']


374       GSM294856
375       GSM294857
376       GSM294858
377       GSM294859
378       GSM294860
379       GSM294861
380       GSM294862
381       GSM294863
382       GSM294864
383       GSM294865
384       GSM294866
385       GSM294867
386       GSM294868
387       GSM294869
388       GSM294870
389       GSM294871
390       GSM294872
391       GSM294873
392       GSM294874
393       GSM294875
394       GSM294876
395       GSM294877
396       GSM294878
397       GSM294879
398       GSM294880
399       GSM294881
400       GSM294882
401       GSM294883
402       GSM294884
403       GSM294885
404       GSM294886
405       GSM294887
406       GSM294888
407       GSM294889
408       GSM294890
409       GSM294891
410       GSM294892
411       GSM294893
3037      GSM452353
3038      GSM452354
3039      GSM452355
3040      GSM452356
3041      GSM452357
3042      GSM452358
3043      GSM452359
3044      GSM452360
3045      GSM452361
3046      GSM452362
3047      GSM452363
3048      GSM452364


## Ткань возраст платформа

In [96]:
series_age.shape, res_series.shape

((51661, 16), (105978, 13))

In [94]:
(
    series_age
    .assign(platform=lambda d: d.platforms.map(','.join))
    .assign(samples_count=1)
    [['platform', 'samples_count']]
    .groupby('platform')
    .samples_count
    .sum()
    .to_frame()
    .reset_index()
    .sort('samples_count', ascending=False)
    [:15]
)

,platform,samples_count
168,GPL570,6870
39,GPL13534,6439
235,GPL8490,2807
7,GPL10558,2754
186,GPL6244,2751
261,GPL96,1529
217,GPL6947,1237
36,GPL13270,966
148,GPL4133,921
196,GPL6480,898


In [ ]:
# cancer_top_ids = 
_t = (
    series_age
    .assign(tissue_id=lambda d: d.tissue_id.map(mapping_top_tissue))
    .assign(age_range=lambda d: pd.cut(d.age, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))
    .assign(_count=lambda d: 1)
    [['series', 'tissue_id', 'age', 'age_range', '_count']]
    .groupby(['tissue_id', 'age_range'])
    .sum()
    .dropna()
    ._count
    .to_frame()
#     .sum()
)

_d = (
    _t
    .reset_index('age_range')
    .drop('age_range', axis=1)
    .groupby(level=0)
    .sum()
    .rename(columns={'_count': '__count'})
)
(
    _t.join(_d)
    .sort(['__count', '_count'], ascending=False)
    .rename(columns={'_count': 'samples_count'})
    .drop('__count', axis=1)
)


### Ткань болезнь платформа

In [99]:
series_tissue_disease_platforms = (
    pd.DataFrame(
        list(db.series.find({'accession': {'$in': series_tissue_disease.series.unique().tolist()}},
                            {'_id': 0, 'accession': 1, 'platforms': 1}))
    )
    .rename(columns={'accession': 'series'})
)

In [106]:
(
    pd.merge(series_tissue_disease, series_tissue_disease_platforms)
    .dropna()
    .assign(platform=lambda d: d.platforms.map(','.join))
    .groupby('platform')
    .samples_count
    .sum()
    .to_frame()
    .sort('samples_count', ascending=False)
#     .platform
#     .value_counts()
#     .to_frame()
)

,samples_count
platform,
GPL570,23574
GPL96,8950
GPL10558,8753
GPL8490,5184
GPL6244,4981
GPL6801,4902
GPL6947,4049
GPL13534,3521
GPL571,2852


### Ткани у которых нет возраста

In [157]:
res_series[:30]

,rid,title,accession,description,age_str,age,detected,detect_crit,is_form_body,is_form_cell_line,datetime,source_name,series
0,513737,Case 8 LNM1,GSM1000061,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GSE36942
1,513738,Case 8 LNM2,GSM1000062,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GSE36942
2,513739,cancer preop 1,GSM1000223,files with prefix 1_ are for Hy3 (Cy3) signals...,41 y,41,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 1",GSE40738
3,513740,cancer preop 2,GSM1000224,files with prefix 1_ are for Hy3 (Cy3) signals...,48 y,48,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 2",GSE40738
4,513741,cancer preop 3,GSM1000225,files with prefix 1_ are for Hy3 (Cy3) signals...,53 y,53,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 3",GSE40738
5,513742,cancer preop 4,GSM1000226,files with prefix 1_ are for Hy3 (Cy3) signals...,54 y,54,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 4",GSE40738
6,513743,cancer preop 5,GSM1000227,files with prefix 1_ are for Hy3 (Cy3) signals...,55 y,55,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 5",GSE40738
7,513744,cancer preop 6,GSM1000228,files with prefix 1_ are for Hy3 (Cy3) signals...,56 y,56,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 6",GSE40738
8,513745,cancer preop 7,GSM1000229,files with prefix 1_ are for Hy3 (Cy3) signals...,56 y,56,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 7",GSE40738
9,513746,cancer preop 8,GSM1000230,files with prefix 1_ are for Hy3 (Cy3) signals...,56 y,56,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 8",GSE40738


In [163]:
series_tissue[:3]

,series,samples,platforms,tissue_id
0,GSE10,"[GSM571, GSM572, GSM573, GSM574]",[GPL4],BTO:0001177
1,GSE10003,"[GSM265549, GSM265550, GSM265551, GSM265552]",[GPL6104],BTO:0000762
2,GSE10007,"[GSM252886, GSM252887, GSM252888, GSM252889]",[GPL2005],BTO:0001573


In [169]:
def cell():
    _t = (
        series_tissue[~(series_tissue.series.isin(res_series.series.unique().tolist()))]
        .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
        .dropna()
        .assign(samples=lambda d: d['samples'].map(len))
        .groupby('tissue_id')
        .samples
        .sum()
        .to_frame()
    )
    return _t

cell()

,samples
tissue_id,
dermis (BTO:0000294),1857
epidermis (BTO:0000404),2485
scalp (BTO:0001809),10
skin (BTO:0001253),4160
skin cancer cell (BTO:0001286),1772
skin fibroblast (BTO:0001255),2652


In [197]:
import pymongo
def cell():
    _t = (
        series_tissue[~(series_tissue.series.isin(res_series.series.unique().tolist()))]
        .assign(tissue_id=lambda d: d.tissue_id.map(mapping_skin))
        .dropna()
        [['series', 'tissue_id']]
#         .assign(samples=lambda d: d['samples'].map(len))
#         .groupby('tissue_id')
#         .samples
#         .sum()
#         .to_frame()
    )
    
    db = pymongo.MongoClient().scraper_series_email
    _q = pd.DataFrame(
        list(db.series.find({'accession': {'$in': _t.series.unique().tolist()}},
                   {'_id': 0, 'accession': 1, 'email': 1})
        )
    ).rename(columns={'accession': 'series'})
    print(_q.shape)
    print(_q.dropna().shape)
#     print(_q[_q.email.isnull()])
    return pd.merge(_t, _q).dropna()

cell().to_csv('../data/skin-emails.csv', index=False)

(539, 2)
(435, 2)


In [198]:
!head ../data/skin-emails.csv

series,tissue_id,email
GSE10123,skin fibroblast (BTO:0001255),scaffidp@mail.nih.gov
GSE10423,skin (BTO:0001253),davewong@stanford.edu
GSE10432,dermis (BTO:0000294),dthiboutot@psu.edu
GSE10433,skin (BTO:0001253),dthiboutot@psu.edu
GSE10434,dermis (BTO:0000294),dthiboutot@psu.edu
GSE10696,epidermis (BTO:0000404),emily.wang@vanderbilt.edu
GSE11181,epidermis (BTO:0000404),jli@med.miami.edu
GSE11307,skin (BTO:0001253),jay.shin@gsc.riken.jp
GSE11550,skin cancer cell (BTO:0001286),jane.kepros.2003@alum.bu.edu


In [194]:
!rm ../datasets-skin-emails.csv